In [1617]:
# Подключаем необходимые библиотеки
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import talib
from plotly import __version__
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [1644]:
# Задаем тикер
ticker = 'NLMK.ME'
# Задаем диапазон дат в котором нужно собирать все данные по тикерам
start = dt.datetime(2010,1,1).date()
#end = dt.datetime(2010,12,31).date()
end = dt.datetime.today().date() # сегодняшняя дата, чтобы не менять вручную.

In [1645]:
# Скачиваем данные
DF = pd.DataFrame(yf.download([ticker],start,end)['Adj Close'])
DF.columns = ['Stock Price']
DF.head

[*********************100%***********************]  1 of 1 completed


<bound method NDFrame.head of             Stock Price
Date                   
2010-03-03    41.328384
2010-03-04    41.042770
2010-03-05    42.287537
2010-03-09    43.903152
2010-03-10    42.986660
...                 ...
2021-06-17   252.591904
2021-06-18   252.106781
2021-06-21   251.350006
2021-06-22   248.740005
2021-06-23   246.399994

[2831 rows x 1 columns]>

In [1646]:
# Задаем индикаторы
DF['RSI'] = talib.RSI(DF['Stock Price'])
DF.dropna(inplace=True)
df = DF.copy()
df.head

<bound method NDFrame.head of             Stock Price        RSI
Date                              
2010-03-24    42.270485  54.988757
2010-03-25    42.202282  54.564311
2010-03-26    43.246681  59.694780
2010-03-29    43.685760  61.655107
2010-03-30    43.272263  58.756825
...                 ...        ...
2017-12-28    90.155075  63.894704
2017-12-29    90.289993  64.261515
2018-01-03    91.381691  67.168109
2018-01-04    93.834877  72.567309
2018-01-05    89.823898  56.272936

[1948 rows x 2 columns]>

In [1647]:
# Задаем стратегию
RSI_treshold = 50
df['positions'] = np.where(df['RSI'] > RSI_treshold,1,0)
df['positions'].sum()

1081

In [1648]:
df

,Stock Price,RSI,positions
Date,,,
2010-03-24,42.270485,54.988757,1
2010-03-25,42.202282,54.564311,1
2010-03-26,43.246681,59.694780,1
2010-03-29,43.685760,61.655107,1
2010-03-30,43.272263,58.756825,1
...,...,...,...
2017-12-28,90.155075,63.894704,1
2017-12-29,90.289993,64.261515,1
2018-01-03,91.381691,67.168109,1


In [1649]:
# df1 = pd.DataFrame()
# df1 = df.copy()
# for i in range(len(df['positions'])):
#     try:
#         if df['positions'][i] < df['positions'][i+1]: #> df['positions'][i+2]:
#             df1['positions'][i+1] = 0
#     except:
#         pass

# df['positions'] = df1['positions']

In [1650]:
#df1['positions'].sum()

In [1651]:
# Визуализация работы стратегии
df[['RSI', 'positions']].iplot(secondary_y = ['positions'], title = "RSI & positions", xTitle = "Data", yTitle = "RSI")
df[['Stock Price', 'positions']].iplot(secondary_y = ['positions'], title = 'Stock Price & positions', xTitle = 'Data', yTitle = 'Stock Price')

In [1652]:
# Расчет доходности
df['Bay_&_hold_return'] = df['Stock Price'].pct_change()
df['RST_strategy_return'] = df['Bay_&_hold_return']*df['positions'].shift(1)

In [1653]:
df['Bay_&_hold_equity'] = (df['Bay_&_hold_return'] + 1).cumprod()
df['RST_strategy_equity'] = (df['RST_strategy_return'] + 1).cumprod()

In [1654]:
# Визуализация доходности
(df[['Bay_&_hold_equity', 'RST_strategy_equity']].iplot(title = "BH VS RSI", xTitle = "Data", yTitle = "Wealth"))

In [1655]:
# Расчет Сортино и максимального падения
df['cum_str_returns_bh'] = (df['Bay_&_hold_return'] + 1).cumprod()
running_max_BH = np.maximum.accumulate(df['cum_str_returns_bh'].dropna())
drawdown_BH = (df['cum_str_returns_bh'])/running_max_BH - 1
max_dd_BH = drawdown_BH.min()*100
df['cum_str_returns_RSI'] = (df['RST_strategy_return'] + 1).cumprod()
running_max_RSI = np.maximum.accumulate(df['cum_str_returns_RSI'].dropna())
drawdown_RSI = (df['cum_str_returns_RSI'])/running_max_BH - 1
max_dd_RSI = drawdown_RSI.min()*100
risk_free_rate = 0.05
trading_days = 252
daily_risk_free_return = risk_free_rate/trading_days
excess_daily_returns_BH = df['Bay_&_hold_return'] - daily_risk_free_return
average_daily_returns_BH = df['Bay_&_hold_return'].mean()
net_returns_BH = df['Bay_&_hold_return'] - average_daily_returns_BH
negative_returns_BH = net_returns_BH[net_returns_BH < 0]
semi_dev_BH = np.sqrt(np.sum((negative_returns_BH**2))/len(df))
sortino_ratio_BH = (excess_daily_returns_BH.mean()/semi_dev_BH) * np.sqrt(trading_days)
excess_daily_returns_RSI = df['RST_strategy_return'] - daily_risk_free_return
average_daily_returns_RSI = df['RST_strategy_return'].mean()
net_returns_RSI = df['Bay_&_hold_return'] - average_daily_returns_RSI
negative_returns_RSI = net_returns_BH[net_returns_RSI < 0]
semi_dev_RSI = np.sqrt(np.sum((negative_returns_RSI**2))/len(df))
sortino_ratio_RSI = (excess_daily_returns_RSI.mean()/semi_dev_RSI) * np.sqrt(trading_days)

In [1656]:
# Выводим результаты
print('With RSI treshold', RSI_treshold)
print('Return from Buy and Hold: ', round(100*(df['Bay_&_hold_equity'][-1]-1),2), '%')
print('Return from RSI strategy: ', round(100*(df['RST_strategy_equity'][-1]-1),2), '%')
print('The Sharpe ratio B&H strategy ', round(np.mean(df['Bay_&_hold_return'])/np.std(df['Bay_&_hold_return'])*(252**0.5),2), '%' )
print('The Sharpe ratio RSI strategy ', round(np.mean(df['RST_strategy_return'])/np.std(df['RST_strategy_return'])*(252**0.5),2), '%' )
print('The Sortino ratio B&H strategy %.2f' % sortino_ratio_BH)
print('The Sortino ratio RSI strategy %.2f' % sortino_ratio_RSI)
print('The maximum drawdown B&H is %.2f' % max_dd_BH)
print('The maximum drawdown RSI is %.2f' % max_dd_RSI)

With RSI treshold 50
Return from Buy and Hold:  112.5 %
Return from RSI strategy:  120.34 %
The Sharpe ratio B&H strategy  0.45 %
The Sharpe ratio RSI strategy  0.56 %
The Sortino ratio B&H strategy 0.45
The Sortino ratio RSI strategy 0.33
The maximum drawdown B&H is -71.73
The maximum drawdown RSI is -50.04
